In [1]:
!pip install evaluate rouge-score > /dev/null 2>&1;

In [2]:
from datasets import load_dataset
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import evaluate
from torch.utils.data import Dataset, DataLoader
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.optim import AdamW
from tqdm import tqdm

In [3]:
rouge=evaluate.load("rouge")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
def compute_rouge_score(sent,ref_summary):
    results=rouge.compute(predictions=[sent], references=[ref_summary])
    return results

In [5]:
answersumm = load_dataset("alexfabbri/answersumm")

README.md:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/24.8M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.76M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2783 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
def generate_summary(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"].to(device), max_length=256, min_length=10, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
train_data=answersumm['train']
inp_dataset=[]
out_dataset=[]
for sample in train_data:
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    inp_str=""
    for ans in sample['answers']:
        for sent in ans['sents']:
            inp_str+=" "+sent['text']
    inp_dataset.append(inp_str)
    out_dataset.append(ref_summ)

In [7]:
class SummarizationDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_input_len=1024, max_target_len=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            max_length=self.max_input_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        targets = self.tokenizer(
            self.summaries[idx],
            max_length=self.max_target_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": targets["input_ids"].squeeze(0),
            "ref_summ": self.summaries[idx],
        }

In [8]:
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
# train_dataset=SummarizationDataset(inp_dataset, out_dataset, tokenizer)
# train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
# optimizer = AdamW(model.parameters(), lr=5e-5)
# epochs=5

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(train_dataloader)}")

In [ ]:
torch.save(model,'BART_FT2.pth')

In [ ]:
# Epoch 1 Loss: 1.1762466367077211

In [9]:
model=torch.load('../input/bart-ft/BART_FT2.pth')

/tmp/ipykernel_30/1732550558.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model=torch.load('../input/bart-ft/BART_FT2.pth')


In [9]:
test_data=answersumm['test']
test_inp_dataset=[]
test_out_dataset=[]
for sample in test_data:
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    inp_str=""
    for ans in sample['answers']:
        for sent in ans['sents']:
            inp_str+=" "+sent['text']
    test_inp_dataset.append(inp_str)
    test_out_dataset.append(ref_summ)

test_dataset=SummarizationDataset(test_inp_dataset,test_out_dataset,tokenizer)
test_dataloader=DataLoader(test_dataset,batch_size=4)

In [10]:
model.eval()
scores=[]
gen_outputs=[]
true_summ=[]
for batch in tqdm(test_dataloader):
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            max_length=256,
            num_beams=4,
            early_stopping=True,
        )
    batch_outputs = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
#     print(batch_outputs[0])
#     print()
#     print(batch['ref_summ'])
    gen_outputs.extend(batch_outputs)
    true_summ.extend(batch['ref_summ'])
for a,b in tqdm(zip(gen_outputs,true_summ)):
    scores.append(compute_rouge_score(a,b))

100%|██████████| 250/250 [35:11<00:00,  8.45s/it]
1000it [02:24,  6.93it/s]


In [11]:
arr=[]
for x in scores:
    arr.append(x['rouge1'])
np.average(arr)

0.16097304840136925

In [12]:
arr=[]
for x in scores:
    arr.append(x['rouge2'])
np.average(arr)

0.04846841487182404

In [13]:
arr=[]
for x in scores:
    arr.append(x['rougeL'])
np.average(arr)

0.10059751991493326